# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 12:30PM,237551,10,MET210207,"Methapharm, Inc.",Released
1,May 18 2022 12:30PM,237551,10,MET210303,"Methapharm, Inc.",Released
2,May 18 2022 12:30PM,237551,10,MET210304,"Methapharm, Inc.",Released
3,May 18 2022 12:30PM,237551,10,MET210305,"Methapharm, Inc.",Released
4,May 18 2022 12:30PM,237551,10,MET210308,"Methapharm, Inc.",Released
5,May 18 2022 12:30PM,237551,10,MET210309,"Methapharm, Inc.",Released
6,May 18 2022 12:30PM,237551,10,MET210312,"Methapharm, Inc.",Released
7,May 18 2022 12:30PM,237551,10,MET210313,"Methapharm, Inc.",Released
8,May 18 2022 12:30PM,237551,10,MET210314,"Methapharm, Inc.",Released
9,May 18 2022 12:30PM,237551,10,MET210315,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
73,237550,Released,1
74,237551,Released,29
75,237552,Released,8
76,237553,Released,1
77,237554,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237550,NaN,NaN,1.0
237551,NaN,NaN,29.0
237552,NaN,NaN,8.0
237553,NaN,NaN,1.0
237554,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237409,4.0,1.0,0.0
237415,0.0,0.0,1.0
237425,0.0,0.0,1.0
237427,0.0,0.0,1.0
237428,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237409,4,1,0
237415,0,0,1
237425,0,0,1
237427,0,0,1
237428,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237409,4,1,0
1,237415,0,0,1
2,237425,0,0,1
3,237427,0,0,1
4,237428,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237409,4,1,
1,237415,,,1
2,237425,,,1
3,237427,,,1
4,237428,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc."
29,May 18 2022 12:30PM,237552,10,"Methapharm, Inc."
37,May 18 2022 12:30PM,237554,10,"Methapharm, Inc."
38,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd"
39,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC
40,May 18 2022 12:22PM,237549,12,"SD Head USA, LLC"
41,May 18 2022 12:19PM,237548,10,Bio-PRF
42,May 18 2022 11:49AM,237547,20,Sartorius Stedim North America
43,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.
64,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc.",,,29
1,May 18 2022 12:30PM,237552,10,"Methapharm, Inc.",,,8
2,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",,,1
3,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",,,1
4,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,,,1
5,May 18 2022 12:22PM,237549,12,"SD Head USA, LLC",,,1
6,May 18 2022 12:19PM,237548,10,Bio-PRF,,,1
7,May 18 2022 11:49AM,237547,20,Sartorius Stedim North America,,,1
8,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,,,21
9,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,,1,24


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc.",29,,
1,May 18 2022 12:30PM,237552,10,"Methapharm, Inc.",8,,
2,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",1,,
3,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",1,,
4,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,1,,
5,May 18 2022 12:22PM,237549,12,"SD Head USA, LLC",1,,
6,May 18 2022 12:19PM,237548,10,Bio-PRF,1,,
7,May 18 2022 11:49AM,237547,20,Sartorius Stedim North America,1,,
8,May 18 2022 11:41AM,237545,10,Eywa Pharma Inc.,21,,
9,May 18 2022 11:40AM,237544,10,Eywa Pharma Inc.,24,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc.",29,,
1,May 18 2022 12:30PM,237552,10,"Methapharm, Inc.",8,,
2,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",1,,
3,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",1,,
4,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc.",29.0,NaN,NaN
1,May 18 2022 12:30PM,237552,10,"Methapharm, Inc.",8.0,NaN,NaN
2,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",1.0,NaN,NaN
3,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 12:30PM,237551,10,"Methapharm, Inc.",29.0,0.0,0.0
1,May 18 2022 12:30PM,237552,10,"Methapharm, Inc.",8.0,0.0,0.0
2,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",1.0,0.0,0.0
3,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4750050,172.0,30.0,40.0
12,475090,2.0,0.0,0.0
15,712532,21.0,0.0,0.0
16,950091,21.0,4.0,0.0
18,4274431,17.0,16.0,0.0
19,1424891,13.0,16.0,6.0
20,1899844,9.0,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4750050,172.0,30.0,40.0
1,12,475090,2.0,0.0,0.0
2,15,712532,21.0,0.0,0.0
3,16,950091,21.0,4.0,0.0
4,18,4274431,17.0,16.0,0.0
5,19,1424891,13.0,16.0,6.0
6,20,1899844,9.0,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,172.0,30.0,40.0
1,12,2.0,0.0,0.0
2,15,21.0,0.0,0.0
3,16,21.0,4.0,0.0
4,18,17.0,16.0,0.0
5,19,13.0,16.0,6.0
6,20,9.0,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,172.0
1,12,Released,2.0
2,15,Released,21.0
3,16,Released,21.0
4,18,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,40.0,0.0,0.0,0.0,0.0,6.0,0.0
Executing,30.0,0.0,0.0,4.0,16.0,16.0,4.0
Released,172.0,2.0,21.0,21.0,17.0,13.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,40.0,0.0,0.0,0.0,0.0,6.0,0.0
1,Executing,30.0,0.0,0.0,4.0,16.0,16.0,4.0
2,Released,172.0,2.0,21.0,21.0,17.0,13.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,40.0,0.0,0.0,0.0,0.0,6.0,0.0
1,Executing,30.0,0.0,0.0,4.0,16.0,16.0,4.0
2,Released,172.0,2.0,21.0,21.0,17.0,13.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()